In [16]:
import grpc

import google.protobuf.duration_pb2

from temporal.api.command.v1.message_pb2 import *
from temporal.api.common.v1.message_pb2 import *
from temporal.api.taskqueue.v1.message_pb2 import *
from temporal.api.workflowservice.v1.request_response_pb2 import *
from temporal.api.workflowservice.v1.service_pb2_grpc import *

In [17]:
wfss = WorkflowServiceStub(grpc.insecure_channel('host.docker.internal:7233'))

# Start Workflow Execution

In [18]:
wfe = wfss.StartWorkflowExecution(
  StartWorkflowExecutionRequest(
    request_id="unique-2", namespace="default", workflow_id="foo/2", workflow_type=WorkflowType(name="Foo"), task_queue=TaskQueue(name="foo")), timeout=60)

In [19]:
wfe.run_id

'621485de-3a9f-4664-876a-3f9749d6c3f0'

# 1st Workflow Task Execution

In [20]:
wft = wfss.PollWorkflowTaskQueue(
  PollWorkflowTaskQueueRequest(
    namespace="default", task_queue=TaskQueue(name="foo")), timeout=60)

In [21]:
wfss.RespondWorkflowTaskCompleted(
  RespondWorkflowTaskCompletedRequest(task_token=wft.task_token, commands=[
    Command(command_type=1, schedule_activity_task_command_attributes=ScheduleActivityTaskCommandAttributes(activity_id="1", activity_type=ActivityType(name="Bar"), task_queue=TaskQueue(name="bar"), start_to_close_timeout=google.protobuf.duration_pb2.Duration(seconds=60)))
    ]), timeout=60)

In [22]:
wft

task_token: "\n$70efea5c-1a42-487b-93b5-a6e4d00c3017\022\005foo/2\032$621485de-3a9f-4664-876a-3f9749d6c3f0 \002(\001"
workflow_execution {
  workflow_id: "foo/2"
  run_id: "621485de-3a9f-4664-876a-3f9749d6c3f0"
}
workflow_type {
  name: "Foo"
}
started_event_id: 3
attempt: 1
backlog_count_hint: 1
history {
  events {
    event_id: 1
    event_time {
      seconds: 1636405780
      nanos: 594538700
    }
    event_type: EVENT_TYPE_WORKFLOW_EXECUTION_STARTED
    task_id: 1048576
    workflow_execution_started_event_attributes {
      workflow_type {
        name: "Foo"
      }
      task_queue {
        name: "foo"
        kind: TASK_QUEUE_KIND_NORMAL
      }
      workflow_task_timeout {
        seconds: 10
      }
      original_execution_run_id: "621485de-3a9f-4664-876a-3f9749d6c3f0"
      first_execution_run_id: "621485de-3a9f-4664-876a-3f9749d6c3f0"
      attempt: 1
      first_workflow_task_backoff {
      }
    }
  }
  events {
    event_id: 2
    event_time {
      seconds: 16364

# 1st Activity Task Execution

In [23]:
act = wfss.PollActivityTaskQueue(
  PollActivityTaskQueueRequest(
    namespace="default", task_queue=TaskQueue(name="bar")), timeout=60)

In [24]:
wfss.RespondActivityTaskCompleted(
  RespondActivityTaskCompletedRequest(
      task_token=act.task_token, result=Payloads(payloads=[Payload(data=b'One')])), timeout=60)

# 2nd Workflow Task Execution

In [25]:
wft = wfss.PollWorkflowTaskQueue(
  PollWorkflowTaskQueueRequest(
    namespace="default", task_queue=TaskQueue(name="foo")), timeout=60)

In [26]:
wfss.RespondWorkflowTaskCompleted(
  RespondWorkflowTaskCompletedRequest(task_token=wft.task_token, commands=[
    Command(command_type=1, schedule_activity_task_command_attributes=ScheduleActivityTaskCommandAttributes(activity_id="2", activity_type=ActivityType(name="Baz-2"), task_queue=TaskQueue(name="baz"), start_to_close_timeout=google.protobuf.duration_pb2.Duration(seconds=60)))
    ]), timeout=60)

# 2nd Activity Task Execution

In [27]:
act = wfss.PollActivityTaskQueue(
  PollActivityTaskQueueRequest(
    namespace="default", task_queue=TaskQueue(name="baz")), timeout=60)

In [28]:
wfss.RespondActivityTaskCompleted(
  RespondActivityTaskCompletedRequest(
      task_token=act.task_token, result=Payloads(payloads=[Payload(data=b'Two')])), timeout=60)

# 3rd Workflow Task Execution

In [29]:
wft = wfss.PollWorkflowTaskQueue(
  PollWorkflowTaskQueueRequest(
    namespace="default", task_queue=TaskQueue(name="foo")), timeout=60)

In [30]:
wfss.RespondWorkflowTaskCompleted(
  RespondWorkflowTaskCompletedRequest(task_token=wft.task_token, commands=[
    Command(command_type=4, complete_workflow_execution_command_attributes=CompleteWorkflowExecutionCommandAttributes(result=Payloads(payloads=[Payload(data=b'OneTwo')])))
    ]), timeout=60)

# Get Workflow Execution History

In [ ]:
wfss.GetWorkflowExecutionHistory(GetWorkflowExecutionHistoryRequest(namespace="default", execution=WorkflowExecution(workflow_id="foo/1", run_id=wfe.run_id)))